In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('fivethirtyeight')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.

### What is [BigQuery](https://cloud.google.com/bigquery/what-is-bigquery)??


Storing and querying massive datasets can be time consuming and expensive without the right hardware and infrastructure. Google BigQuery is an enterprise data warehouse that solves this problem by enabling super-fast SQL queries using the processing power of Google's infrastructure. Simply move your data into BigQuery and let us handle the hard work. You can control access to both the project and your data based on your business needs, such as giving others the ability to view or query your data.

You can access BigQuery by using a [web UI ](https://bigquery.cloud.google.com/project/nice-particle-195309)or a [command-line tool](https://cloud.google.com/bigquery/bq-command-line-tool), or by making calls to the BigQuery REST API using a variety of client libraries such as Java, .NET, or Python. There are also a variety of third-party tools that you can use to interact with BigQuery, such as visualizing the data or loading the data.
Because the datasets on BigQuery can be very large, there are some restrictions on how much data you can access. 

*But You dont need to go to Google, Since Kaggle kernels allows you to access TeraBytes of data from Google cloud with all saftey measures like not letting your query go above memory limits and helper APIs. Thanks to [Sohier](https://www.kaggle.com/sohier)'s [BigQuery helper module](https://github.com/SohierDane/BigQuery_Helper/blob/master/bq_helper.py).
Each Kaggle user can scan 5TB every 30 days for free.*

Let's first setup environment to run BigQueries in Kaggle Kernels.

### Importing Kaggle's bq_helper package

In [ ]:
import bq_helper 

### Creating a helper object for  bigquery dataset

The addresses of BigQuery datasets look like this![](https://i.imgur.com/l11gdKx.png)

for us dataset is **github_repos**

[Rachael](https://www.kaggle.com/rtatman) from Kaggle has ran a 5 days BigQuery Introductory challenge called SQL Scavenger Hunt. We will go through day 1 to 5 using Github Repos Dataset.

Image is taken from [SQL Scavenger Handbook](https://www.kaggle.com/rtatman/sql-scavenger-hunt-handbook)

In this case we are going to derive data from the monthy github archive datasets

In [ ]:
github_archive = bq_helper.BigQueryHelper(active_project= "githubarchive", 
                                       dataset_name = "month")

github_archive_day = bq_helper.BigQueryHelper(active_project= "githubarchive", 
                                       dataset_name = "day")

### Listing Tables

In [ ]:
# print a list of all the tables in the github archive dataset
github_archive.list_tables()

###  Printing Table Schema


In [ ]:
# print information on all the columns in the september 2015 dataset
# in the github_archive dataset
github_archive.table_schema("201509")

In [ ]:
# preview the first couple lines of the table
github_archive.head("201509")

### Checking the size of our query before we run it



Our Dataset is quite large so we can easily cross tha monthly limit by running few queries. 

We should always estimate  how much data we need to scan for executing this query by **BigQueryHelper.estimate_query_size()** method.


In [ ]:
query_pulls= """SELECT
            repo,
            type,
            actor,
            payload
            FROM `githubarchive.month.201509`
            WHERE type = 'PullRequestEvent'
        """

query_pulls_user= """SELECT
            repo.name as repo_name,
            JSON_EXTRACT(payload, '$.pull_request.user.login') as user,
            JSON_EXTRACT(payload, '$.pull_request.base.repo.language') as language
            FROM `githubarchive.day.20150901`
            WHERE type = 'PullRequestEvent'
            LIMIT 1000
        """


query_commits_user= """SELECT
            repo.name as repo_name,
            UNNEST(JSON_EXTRACT(payload, '$.commits')) as user,
            FROM `githubarchive.day.20150901`
            WHERE type = 'PushEvent'
            LIMIT 1000
        """

query_commits_user= """SELECT
            repo.name as repo_name,
            JSON_EXTRACT(payload, '$.commits') as commits,
            actor.login as user_pull
            FROM `githubarchive.day.20150901`
            WHERE type = 'PushEvent'
            LIMIT 1000
        """


github_archive.estimate_query_size(query_commits_user)

In [ ]:
github_repo_commits = github_archive.query_to_pandas_safe(query_commits_user, max_gb_scanned=2)
github_repo_commits.head()

In [ ]:
github_repo_commits.to_csv('github_repo_commits_20150901.csv',index=False)

In [ ]:
query1= """SELECT
            repo,
            JSON_EXTRACT(payload, '$.pull_request.user.login') as puller,
            JSON_EXTRACT(payload, '$.pull_request.base.repo.language') as language
            FROM `githubarchive.month.201501`
            WHERE type = 'PullRequestEvent' AND JSON_EXTRACT(payload, '$.action.repo.language')
         """

query_pulls_user= """SELECT
            repo.name as repo_name,
            JSON_EXTRACT(payload, '$.pull_request.user.login') as user,
            JSON_EXTRACT(payload, '$.pull_request.base.repo.language') as language
            FROM `githubarchive.month.201501`
            WHERE type = 'PullRequestEvent'
        """

query_pulls_user_grouped= """SELECT
            JSON_EXTRACT(payload, '$.pull_request.user.login') AS user,
            repo.name AS repo_name,
            JSON_EXTRACT(payload, '$.pull_request.base.repo.language') AS language,
            COUNT(*) AS number_actions
            FROM `githubarchive.day.20150901`
            WHERE type = 'PullRequestEvent'
            GROUP BY repo_name, user, language
        """

# Where clause not working, dont know why
query_pulls_user_grouped_opened= """SELECT
            JSON_EXTRACT(payload, '$.pull_request.user.login') AS user,
            repo.name AS repo_name,
            JSON_EXTRACT(payload, '$.pull_request.base.repo.language') AS language,
            COUNT(*) AS number_actions
            FROM `githubarchive.day.20150901`
            WHERE ((type = 'PullRequestEvent') AND (JSON_EXTRACT(payload, '$.action') = 'opened'))
            GROUP BY repo_name, user, language
        """

# This is the definitive query
query_pulls_user_grouped= """SELECT
            JSON_EXTRACT(payload, '$.pull_request.user.login') AS user,
            repo.name AS repo_name,
            JSON_EXTRACT(payload, '$.pull_request.base.repo.language') AS language,
            JSON_EXTRACT(payload, '$.action') AS action,
            JSON_EXTRACT(payload, '$.pull_request.merged') AS merged,
            COUNT(*) AS number_actions
            FROM `githubarchive.day.20150901`
            WHERE type = 'PullRequestEvent'
            GROUP BY repo_name, user, language, action, merged
        """



github_archive_day.estimate_query_size(query_pulls_user_grouped)

Now I will use the query to pandas function to make the query of the dayly dataframe as a test

In [ ]:
github_user_pulls = github_archive_day.query_to_pandas_safe(query_pulls_user_grouped, max_gb_scanned=2)
github_user_pulls.head()

In [ ]:
github_user_pulls.to_csv('github_user_pull_20150901_actions.csv',index=False)

Now I want to make a for loop that changes the query, extracting data from month to month during an specific year


In [ ]:
import numpy as np 
import pandas as pd 
import bq_helper

# Github archive dataset
github_archive = bq_helper.BigQueryHelper(active_project= "githubarchive", dataset_name = "month")

#Arrays with the months and years under study
years_array = ["2015","2016","2017","2018","2019"]
months_array = ["01", "02", "03", "04", "05", "06", "07","08", "09", "10", "11","12"]

# Loop through the years and months
for query_year in years_array:           
    for query_month in months_array:
        
        # Print the year and month
        print("year = " + query_year + ", month = " + query_month)
        
        # Makes the query, it will change the database according to year and month
        query_pull_user= """SELECT
                    actor.login as user,
                    repo.name AS repo_name,
                    COUNT(*) AS number_actions
                    FROM `githubarchive.month.{year}{month}`
                    WHERE type = 'PushEvent'
                    GROUP BY repo_name, user
                """.format(year= query_year,month=query_month)

        
        # Executes the query and saves it to a mandas datafram, it will not work if more than 250 gb are scanned
        github_user_pulls = github_archive.query_to_pandas_safe(query_pull_user, max_gb_scanned=250)
        
        # Writes the csv
        github_user_pulls.to_csv("github_user_push_" + query_year + query_month + ".csv",index=False)
        
        print("Ready!")
    

In [ ]:
    # Query for the extraction of the number of pull actions in the dataframe
    query_push_user= """SELECT
                actor.login as user
                repo.name AS repo_name,
                COUNT(*) AS number_actions
                FROM `githubarchive.month.{year}{month}`
                WHERE type = 'PushEvent'
                GROUP BY repo_name, user
            """.format(year= query_year,month=query_month)
    

### How many github repositories are in form of binary files?
A binary file is a file stored in binary format. A binary file is computer-readable but not human-readable. All executable programs are stored in binary files, as are most numeric data files.

In [ ]:
#%%time
#query2= """SELECT binary
#            FROM `bigquery-public-data.github_repos.contents`
#            LIMIT 50000
#        """
#
#binary_files=github_repos.query_to_pandas_safe(query2)